In [1]:
import numpy as np
from scipy import linalg

# 14.1

## Задані виміри економіки країни

In [2]:
X = np.array([
    [1320, 1170],
    [1060, 965]
])

In [3]:
y = np.array([
    [1075],
    [1185]
])

In [4]:
s = np.array([0.45, 0.2])

# Модель міжгалузевої залежності цін

Нехай $X$ - це матриця в якій $x_{i j}$ елемент позначає витрати продукції $i$-ї галузі на потреби $j$-ї.

y - вектор $y_i$ елемент якого це обсяг $i$-ї продукції, що витрачається у невиробничій сфері.

Тоді загальний випуск $i$-ї галузі (позначимо $x_i$), при загальній кількості галузей $n$, можна записати як:
\begin{equation}
x_i = \sum_{j=1}^n x_{i j} + y_i
\label{eq:sum_prod}
\tag{1}
\end{equation}

У моделі Леонтьєва вводиться припущення про пропорційну залежність між витратами та об'ємом виробництва, тобто вводяться лінійно-однорідні функції виробничих витрат:
\begin{equation}
x_{i j}=a_{i j} x_j => a_{i j} = \frac{x_{i j}}{ x_j}
\label{eq:leon_assum}
\tag{2}
\end{equation}

Модель міжгалузевої залежності цін описується рівнянням:
\begin{equation}
p = A^T p + s => (E - A ^ T) p = s
\label{eq:pr_dep}
\tag{3}
\end{equation}
Тут $p$ - вектор цін на товари, $s$ - вектор доданих вартостей в ціні

In [5]:
x = (np.sum(X, axis=1).reshape(-1, 1) + y)
print(x)

[[3565]
 [3210]]


In [6]:
A = X / x.T
print(A)

[[ 0.37026648  0.36448598]
 [ 0.2973352   0.30062305]]


In [7]:
M = np.eye(A.shape[0]) - A.T

In [8]:
p = np.linalg.solve(M, s)
print("Ціни на промислову та сільськогосподарську продукцію: {}".format(p))

Ціни на промислову та сільськогосподарську продукцію: [ 1.12691012  0.87326719]


# 14.2

In [9]:
A = np.array([
    [0.7, 0.05, 0.05],
    [0.05, 0.75, 0.05],
    [0.6, 0.15, 0.05]
])
y = np.array([
    [1900],
    [1500],
    [1100]
])

## Аналіз матриці прямих виробничих витрат $A$

In [10]:
w = linalg.eig(A, left=True)

In [11]:
eigvalues = w[0]
pol_coef = np.poly(w[0])
max_eig_ind = np.argmax(w[0])
fro_number = w[0][max_eig_ind]
fro_l = w[1][:, max_eig_ind] * (-1 if w[1][0, max_eig_ind] < 0 else 1)
fro_r = w[2][:, max_eig_ind] * (-1 if w[2][0, max_eig_ind] < 0 else 1)

In [12]:
print("Характеристичний поліном: {}".format(pol_coef))
print("Власні числа: {}".format(", ".join(map(lambda x: str(x.real), eigvalues))))
print("Число Фробеніуса: {}".format(fro_number.real))
print("Лівий вектор Фробеніуса: ({})".format(", ".join(map(lambda x: str(x.real), fro_l))))
print("Правий вектор Фробеніуса: ({})".format(", ".join(map(lambda x: str(x.real), fro_r))))

Характеристичний поліном: [  1.00000000e+00  -1.50000000e+00   5.57500000e-01  -2.50000000e-04]
Власні числа: 0.000448972689168, 0.676723625695, 0.822827401615
Число Фробеніуса: 0.8228274016154079
Лівий вектор Фробеніуса: (0.723483833587, 0.684306570005, 0.0910805181499)
Правий вектор Фробеніуса: (0.494997713423, 0.69649905063, 0.519486608276)


## Продуктивність

Теорема
Для продуктивності моделі Леонтьєва необхідно й достататньо, щоб фробеніусове число $\lambda_A$ матриці $A$ задовольняло нерівності $\lambda_A < 1$

In [13]:
print("Матриця A - {}продуктивна".format("" if fro_number.real < 1 else "не "))

Матриця A - продуктивна


## Матриця повних витрат

Матриця повних витрат $B$ обчислюється як сума ряду:
\begin{align}
B &= E + A + A^2 + ... + A^N \\
N &\rightarrow \inf
\end{align}
Для того, щоб дослідити ряд на збіжність та обчислити матрицю B побудуємо послідовність $B_i$, яка прямує до B:
\begin{align}
&B_0 = E \\
&B_{i + 1} = B_i A + E
\end{align}
Тоді критерій збіжності набуває вигляду:
$$max( \bigl| B_i - B_{i + 1} \bigr| ) < \epsilon$$

In [14]:
B = np.eye(A.shape[0])
B_pr = B.copy()
eps = 0.01
for i in range(100):
    B_pr, B = B, B_pr
    B = np.dot(B_pr, A) + np.eye(A.shape[0])
    if np.max(np.fabs(B - B_pr)) < eps:
        break

In [15]:
print("Матриця повних витрат")
print(B)

Матриця повних витрат
[[ 3.98575637  0.93122482  0.25804895]
 [ 1.30988624  4.41209929  0.30012244]
 [ 2.71792601  1.27902876  1.26222235]]


## Вектор кінцевого випуску

З моделі Леонтьєва:
$$x = A x + y => (E-A) x = y$$
Розвязавши систему лінійних рівнянь отримуємо вектор кінцевого випуску $x$

In [16]:
x = np.linalg.solve(np.eye(A.shape[0]) - A, y)

In [17]:
x

array([[ 9362.44541485],
       [ 9589.51965066],
       [ 8585.15283843]])